# Sentence Quality Scorer

In [1]:
!pip install textacy
!python3 -m spacy download en
import textacy
import pandas as pd
from textacy.text_stats import TextStats
print("Loaded")

    100% |████████████████████████████████| 143kB 2.5MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 3.0MB/s ta 0:00:011
    100% |████████████████████████████████| 31.2MB 21kB/s  eta 0:00:01
    100% |████████████████████████████████| 12.4MB 54kB/s  eta 0:00:01   48% |███████████████▋                | 6.0MB 54.9MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 5.8MB/s eta 0:00:01
    100% |████████████████████████████████| 2.0MB 359kB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 5.7MB/s eta 0:00:01
    100% |████████████████████████████████| 1.6MB 438kB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 2.8MB/s eta 0:00:01
  Running setup.py bdist_wheel for ftfy ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/37/54/00/d320239bfc8aad1455314f302dd82a75253fc585e17b81704e
  Running setup.py bdist_wheel for pyemd ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/e4/ba/b0/1f4178a35c916b22fc51dc56f278

  Running setup.py install for en-core-web-sm ... done
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /srv/conda/lib/python3.6/site-packages/en_core_web_sm -->
    /srv/conda/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')

Loaded


In [2]:
df = pd.read_csv("./abstraction_scored.csv")
df.clauses_text_final = df.clauses_text_final.apply(eval)
df.voice = df.voice.apply(eval)
df.abstraction_score = df.abstraction_score.apply(eval)
df.sample(frac = 1).head(10)

,survey_id,prompt_number,prompt_id,prompt,response,clauses_text_final,voice,idx,abstraction_score
7,3127,29,29,If my mother,"was a different lady, I would not be who I am.","[was a different lady, I would not be, who I am]","[P_bevb_x, A_def, P_bevb_x]",7,"[0.14, 0.22, 0.22]"
17,2157,17,17,When they avoided me,hmmm... I'm stumped. I have no idea how to res...,"[hmmm, I m, I have no idea how to respond to t...","[Undefined, A_def, A_def, A_def, P_bevb_x]",17,"[0.14, 0.22, 0.22, 0.14]"
23,2952,25,25,My main problem is,getting over decades spent proving myself.,"[getting over decades, spent proving myself]","[A_def, A_def, A_def]",23,"[0.25, 0.14]"
19,2046,10,10,When people are helpless,I am inspired to reach out and be of service.,"[I am inspired to reach out and, be of service]","[P_bevb_x, A_def, A_def]",19,"[0.25, 0.25]"
6,1869,25,25,My main problem is,a feeling that I am not entitled,"[a feeling, that I am not entitled]","[Undefined, P_bevb_x]",6,"[0.14, 0.22]"
9,2390,17,17,When they avoided me,"I felt safe, knowing that I could receive a wa...","[I felt safe, that I could receive a warm welc...","[P_get_x, A_pron_x]",9,"[0.22, 0.25]"
24,2721,16,16,I feel sorry,for the pain that all humans and all beings on...,"[for the pain, that all humans and all beings ...","[Undefined, A_def, A_def]",24,"[0.14, 0.25, 0.14]"
29,2374,11,39,What I like to do best is,be a conduit. There is no I.,"[be a conduit, There is no I]","[A_def, P_bevb_x]",29,"[0.25, 0.22]"
35,3181,12,12,A good boss,is a solar being ... one that brings light and...,"[is a solar, being one, that brings light and ...","[P_bevb_x, A_def, A_def, A_def, P_bevb_x, A_pr...",35,"[0.12, 0.14, 0.14, 0.25, 0.14, 0.25, 0.14, 0.25]"
48,1889,15,41,Privacy,the human formconsicousness separates from the...,[the human formconsicousness separates from th...,[A_def],48,[0.25]


In [3]:
def score_readability(text):
    doc = textacy.Doc(text, lang = "en")
    ts = TextStats(doc)
    #print("############################################################")
    #print(text)
    #print("readability_stats = ", ts.readability_stats)
    return ts.readability_stats

df['readability_attributes_score'] = df.clauses_text_final.apply(lambda arr: [score_readability(x) for x in arr])
df['grading_level'] = df.readability_attributes_score.apply(lambda dct_arr: [round(dct['flesch_kincaid_grade_level'], 2) for dct in dct_arr])
df['reading_ease'] = df.readability_attributes_score.apply(lambda dct_arr: [round(dct['flesch_reading_ease'], 2) for dct in dct_arr])
_ = """[{'flesch_kincaid_grade_level': 0.6257142857142846, 'flesch_reading_ease': 103.04428571428573, 'smog_index': 3.1291, 'gunning_fog_index': 2.8000000000000003, 'coleman_liau_index': 2.6518669999999993, 'automated_readability_index': 0.23714285714285666, 'lix': 7.0, 'gulpease_index': 93.28571428571428, 'wiener_sachtextformel': -2.5074571428571426}]"""
del df['readability_attributes_score']
df.sample(frac = 1).head(10)

,survey_id,prompt_number,prompt_id,prompt,response,clauses_text_final,voice,idx,abstraction_score,grading_level,reading_ease
44,1823,3,3,Change is,the heartbeat of the universe... aliveness cal...,"[the heartbeat of the universe aliveness, call...","[Undefined, A_def]",44,"[0.14, 0.25]","[4.45, 1.31]","[73.85, 90.99]"
35,3181,12,12,A good boss,is a solar being ... one that brings light and...,"[is a solar, being one, that brings light and ...","[P_bevb_x, A_def, A_def, A_def, P_bevb_x, A_pr...",35,"[0.12, 0.14, 0.14, 0.25, 0.14, 0.25, 0.14, 0.25]","[1.31, 2.89, 2.31, 13.07, -1.84, 6.62, -0.67, ...","[90.99, 77.91, 90.96, 28.05, 117.16, 54.73, 11..."
24,2721,16,16,I feel sorry,for the pain that all humans and all beings on...,"[for the pain, that all humans and all beings ...","[Undefined, A_def, A_def]",24,"[0.14, 0.25, 0.14]","[-2.62, 7.0, 2.89]","[119.19, 75.12, 77.91]"
21,2381,34,47,Technology,"is a great tool, nice to have and can bring gl...","[is a great tool nice, to have and, can bring ...","[P_bevb_x, A_def, A_def, A_def, P_bevb_x, A_de...",21,"[0.25, 0.14, 0.25, 0.14, 0.12, 0.22, 0.14]","[-1.84, -2.62, 9.66, 9.18, 6.42, 5.24, 0.52]","[117.16, 119.19, 40.09, 34.59, 59.75, 66.4, 10..."
14,1529,10,10,When people are helpless,At times I try to find other ways of doing thi...,"[At times, I try to find other ways of, doing ...","[Undefined, A_def, A_def, A_def, A_def, A_def,...",14,"[0.11, 0.22, 0.14, 0.22, 0.12]","[-3.01, 0.63, 2.89, 5.68, -2.62]","[120.21, 103.04, 77.91, 66.79, 119.19]"
12,3233,5,5,Being with other people,is both a learning experience and an opportuni...,[is both a learning experience and an opportun...,"[P_bevb_x, A_def]",12,"[0.25, 0.25]","[11.13, 1.31]","[29.52, 90.99]"
9,2390,17,17,When they avoided me,"I felt safe, knowing that I could receive a wa...","[I felt safe, that I could receive a warm welc...","[P_get_x, A_pron_x]",9,"[0.22, 0.25]","[-2.62, 4.82]","[119.19, 81.86]"
37,2619,2,2,When I am criticized,I can embody my personality and feel a pang co...,"[I can embody my personality and, feel a pang ...","[A_pron_x, A_def, A_def, A_def, A_def, A_def, ...",37,"[0.25, 0.25, 0.22, 0.12, 0.14, 0.25]","[8.38, 2.28, 4.45, 0.72, 3.65, 5.86]","[45.65, 92.97, 73.85, 97.03, 86.71, 72.62]"
39,1909,25,25,My main problem is,"integrating emptiness with everyday life, find...","[integrating emptiness with making, everyday l...","[A_def, A_def, A_def]",39,"[0.14, 0.14, 0.14]","[12.52, 10.35, 7.6]","[12.43, 31.55, 49.48]"
25,2276,30,30,If I were in charge,i would change that by facilitating change tha...,"[i would change that by, facilitating change i...","[A_def, A_pron_x, A_pron_x, A_def, P_bevb_x]",25,"[0.25, 0.25, 0.25, 0.25, 0.14]","[-1.84, 8.37, 9.82, 3.67, 0.63]","[117.16, 52.87, 53.12, 75.88, 103.04]"


In [4]:
def normalize(row, x_max, x_min, reverse_arr = False):
    if not reverse_arr:
        return [round((x - x_min)/(x_max - x_min), 2) for x in row]
    return [round((-1*x - x_min)/(x_max - x_min), 2) for x in row]

reading_ease = df['reading_ease'].tolist()
reading_ease = [j for i in reading_ease for j in i]
reading_ease = [-1*x for x in reading_ease]
x_max, x_min = max(reading_ease), min(reading_ease)
df['reading_ease_normalized'] = df['reading_ease'].apply(lambda arr : normalize(arr, x_max, x_min, reverse_arr = True))

grading_levels = df['grading_level'].tolist()
grading_levels = [j for i in grading_levels for j in i]
x_max, x_min = max(grading_levels), min(grading_levels)
df['grading_level_normalized'] = df['grading_level'].apply(lambda arr : normalize(arr, x_max, x_min, reverse_arr = False))
df.sample(frac = 1).head(10)

,survey_id,prompt_number,prompt_id,prompt,response,clauses_text_final,voice,idx,abstraction_score,grading_level,reading_ease,reading_ease_normalized,grading_level_normalized
41,1805,4,37,"These days, work",Is a rainbow of consciousness manifesting thro...,"[a rainbow of consciousness, manifesting throu...","[Undefined, A_def, A_def]",41,"[0.25, 0.25, 0.25]","[6.62, 12.32, -2.62]","[54.73, 15.64, 119.19]","[0.31, 0.5, 0.01]","[0.34, 0.53, 0.03]"
2,2510,33,33,When I am nervous,my hands get shaky,[my hands get shaky],[A_pron_x],2,[0.14],[-2.23],[118.18],[0.01],[0.04]
27,1883,1,1,Raising a family,is a profound gift. I am tending the future. I...,"[is a profound gift, I am tending the future, ...","[P_bevb_x, P_bevb_x, P_bevb_x, A_def, P_bevb_x...",27,"[0.14, 0.22, 0.22, 0.25, 0.25, 0.14]","[0.72, 2.88, 1.31, 5.25, -1.84, 2.31]","[97.03, 83.32, 90.99, 62.79, 117.16, 90.96]","[0.11, 0.18, 0.14, 0.27, 0.02, 0.14]","[0.14, 0.21, 0.16, 0.29, 0.05, 0.19]"
8,1832,26,26,When I get mad,"it takes a long time to build up, but I am slo...","[it takes a long time but, to build up, I am s...","[A_def, A_def, P_bevb_x, A_def]",8,"[0.14, 0.12, 0.22]","[-1.45, -2.62, 0.52]","[116.15, 119.19, 100.24]","[0.02, 0.01, 0.1]","[0.07, 0.03, 0.13]"
37,2619,2,2,When I am criticized,I can embody my personality and feel a pang co...,"[I can embody my personality and, feel a pang ...","[A_pron_x, A_def, A_def, A_def, A_def, A_def, ...",37,"[0.25, 0.25, 0.22, 0.12, 0.14, 0.25]","[8.38, 2.28, 4.45, 0.72, 3.65, 5.86]","[45.65, 92.97, 73.85, 97.03, 86.71, 72.62]","[0.36, 0.13, 0.22, 0.11, 0.16, 0.23]","[0.39, 0.19, 0.26, 0.14, 0.24, 0.31]"
1,2507,1,90,My family,has a cat...I don't have a cat though I have 2...,"[has a cat, I don t, have a cat, though I have...","[A_def, A_def, A_def, A_def]",1,"[0.25, 0.22, 0.25, 0.22]","[-2.62, -2.62, -2.62, -1.84]","[119.19, 119.19, 119.19, 117.16]","[0.01, 0.01, 0.01, 0.02]","[0.03, 0.03, 0.03, 0.05]"
17,2157,17,17,When they avoided me,hmmm... I'm stumped. I have no idea how to res...,"[hmmm, I m, I have no idea how to respond to t...","[Undefined, A_def, A_def, A_def, P_bevb_x]",17,"[0.14, 0.22, 0.22, 0.14]","[8.4, -3.01, 1.03, 2.31]","[36.62, 120.21, 103.7, 90.96]","[0.4, 0.0, 0.08, 0.14]","[0.39, 0.01, 0.15, 0.19]"
34,2215,13,40,We could make the world a better place if,"the mystery was okay to be as a question, that...","[the mystery was okay to be as a question, tha...","[P_bevb_x, A_def, P_bevb_x, A_def, P_bevb_x, A...",34,"[0.14, 0.14, 0.14, 0.25, 0.25, 0.25, 0.12, 0.1...","[2.34, -2.23, 2.47, 2.31, 5.23, 2.48, 5.24, -1...","[94.3, 118.18, 95.17, 90.96, 71.82, 87.95, 66....","[0.13, 0.01, 0.12, 0.14, 0.23, 0.16, 0.26, 0.0...","[0.19, 0.04, 0.2, 0.19, 0.29, 0.2, 0.29, 0.05,..."
21,2381,34,47,Technology,"is a great tool, nice to have and can bring gl...","[is a great tool nice, to have and, can bring ...","[P_bevb_x, A_def, A_def, A_def, P_bevb_x, A_de...",21,"[0.25, 0.14, 0.25, 0.14, 0.12, 0.22, 0.14]","[-1.84, -2.62, 9.66, 9.18, 6.42, 5.24, 0.52]","[117.16, 119.19, 40.09, 34.59, 59.75, 66.4, 10...","[0.02, 0.01, 0.38, 0.41, 0.29, 0.26, 0.09]","[0.05, 0.03, 0.44, 0.42, 0.33, 0.29, 0.13]"
23,2952,25,25,My main problem is,getting over decades spent proving myself.,"[getting over decades, spent proving myself]","[A_def, A_def, A_def]",23,"[0.25, 0.14]","[1.31, 5.25]","[90.99, 62.79]","[0.14, 0.27]","[0.16, 0.29]"


In [5]:
#Cross verify that they are correct
reading_ease = df['reading_ease_normalized'].tolist()
reading_ease = [j for i in reading_ease for j in i]
grade = df['grading_level_normalized'].tolist()
grade = [j for i in grade for j in i]
print(max(reading_ease), min(reading_ease), max(grade), min(grade))
df[["prompt", "response", "clauses_text_final", "voice", "idx", "abstraction_score", "reading_ease_normalized", "grading_level_normalized"]].to_csv("readability_scored.csv", index = False)

1.0 0.0 1.0 0.0


In [21]:
from textacy.keyterms import sgrank
df['nlp_doc'] = df.apply(lambda row : textacy.Doc(row['prompt'] + " " + row['response'], lang = "en"), axis = 1)
df['sgrank'] = df['nlp_doc'].apply(lambda doc : sgrank(doc, n_keyterms = len(doc)))

def get_normalized_importance(df):
    clauses = df["clauses_text_final"]
    rank_tuples = dict(df['sgrank'])
    ngram_keys = rank_tuples.keys()
    op = []
    for clause in clauses:
        str_clause = "".join(clause)
        denominator = 0
        numerator = 0
        for x in ngram_keys:
            if x in str_clause:
                numerator += rank_tuples[x]
                denominator += 1
        op.append(round(numerator / denominator, 2) if denominator > 0 else 0.0)
    return op
    
df["sgrank_normalized"] = df.apply(get_normalized_importance, axis = 1)
df[["prompt", "response", "clauses_text_final", "voice", "idx", "abstraction_score", "reading_ease_normalized", "grading_level_normalized", "sgrank_normalized"]].to_csv("keyterm_scored.csv", index = False)